## Creating 2 random matrices

In [ ]:
import random

#creating 2 matrices of random numbers between start and stop(0,9)  
start = 0
stop = 9
#size of each matrix nXn
n = 2
all_elements = n*n

#fisrt matrix with one digit random numbers
first_matrix = [random.randint(start, stop) for iter in range(all_elements)]

#second matrix with one digit random numbers
second_matrix =  [random.randint(start, stop) for iter in range(all_elements)]

#print(first_matrix)
#print(second_matrix)

## Writing randomly generated matrices to files A.txt and B.txt

In [ ]:
matrices = {"A.txt":first_matrix, "B.txt":second_matrix}

for item in matrices.items():
    textfile = open(item[0], "w")
    i =0
    for element in item[1]:
        i += 1
        textfile.write(str(element))
        
        if i >= n and i%int(n) == 0 :
            textfile.write("\n")
        else:
            textfile.write(" ")
    textfile.close()

## Creating pyspark context and read txt files into rdd files

In [ ]:
from pyspark import SparkContext

#create Spark context with necessary configuration
sc = SparkContext.getOrCreate()

indexed_matrices=[]

for item in matrices.items():
    #read data from text file and split each line into one element
    rddfile_1 = sc.textFile(item[0]).map(lambda line: line.split(" "))
    #convert all elements of the array from string to int 
    indexed_matrices.append(rddfile_1.map(lambda x: [int(ele) for ele in x]))

## Creating lists in the format assignment offered for matrices

In [ ]:
#first matrix format((i,k)(M,j,a[i][j]))
results = []
matrix_values = indexed_matrices[0].collect()
def firstMatrixAssignValues(value,j,k,x):
    i = matrix_values.index(x)
    return ((i,k), (0,j,value))

results.append(indexed_matrices[0].map(lambda x : [firstMatrixAssignValues(value,index,k,x) for index, value in enumerate(x) for k in range(n)]))

#results[0].collect()

In [ ]:
#second matrix format((i,k)(M,j,a[i][j]))
matrix2_values = indexed_matrices[1].collect()
def secondMatrixAssignValues(value,k,i,x):
    j = matrix2_values.index(x)
    return ((i,k), (1,j,value))

results.append(indexed_matrices[1].map(lambda x : [secondMatrixAssignValues(value,index,i,x) for index, value in enumerate(x) for i in range(n)]))

#results[1].collect()

In [ ]:
#merging results of the mapped matrices into one rdd
final = results[1] + results[0]

#final.collect()

In [ ]:
flatmap_result =  final.flatMap(lambda x: x)

#flatmap_result.collect()

In [ ]:
#reduce the elements that have the same keys to one
reduced_res = flatmap_result.reduceByKey(lambda x, y: (x+y)).sortByKey()

#reduced_res.collect()

In [ ]:
#put the values of each key in array format
reduced_res2 = reduced_res.map(lambda x: (x[0],list(x[1])))

#reduced_res2.collect()

In [ ]:
#seperate each 3 elements of the values array into multiple arrays of length 3
reduced_res3 = reduced_res2.map(lambda x: (x[0],[x[1][i:i+3] for i in range(0, len(x[1]), 3)]))

#reduced_res3.collect()

In [ ]:
#collect the value of reduced_res3 rdd
list_p = reduced_res3.collect()

#print(list_p)

## Calculate the elemets in matrix C which is the result of AXB and store it in an array

In [ ]:
#saerch for the lists of values that have same j value to multiply them to each other and store it in a dictionary
#then sum up the values to calculate the final result for C matrix which is the result of AXB 
final_result = []

for j in range(n*n):
    lookup_for_match = list_p[j][1]
    
    #print(lookup_for_match)
    B = {}
    for li in lookup_for_match:
        if not li[1] in B:
             B[li[1]] = 1
        B[li[1]] *= li[2]

    #print(B)
    sum = 0
    for i in B.values():
        sum+=i
    
    #print(sum)
    final_result.append(sum)

    #print(final_result)

## Writing the result of the multiplication into "C.txt" file

In [ ]:
textfile = open("C.txt", "w")
   
i =0
for element in final_result:
    i += 1
    textfile.write(str(element))
    if i >= n and i%int(n) == 0 :
         textfile.write("\n")
    else:
         textfile.write(" ")
textfile.close()